In [164]:
import hvplot.pandas
import pandas as pd
import requests
from api_keys import geoapify_key
import os
import json 
from pprint import pprint

In [165]:
city_data_df = pd.read_csv("output/cities.csv")
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,24.11,91,100,3.32,BR,1675307374
1,1,bluff,-46.6000,168.3333,17.98,92,85,5.69,NZ,1675306989
2,2,ribeira grande,38.5167,-28.7000,12.10,81,20,1.79,PT,1675307010
3,3,hobart,-42.8794,147.3294,18.92,48,75,9.26,AU,1675306971
4,4,mataura,-46.1927,168.8643,22.04,76,100,1.91,NZ,1675306959
...,...,...,...,...,...,...,...,...,...,...
579,579,kot isa khan,30.9500,75.1333,10.50,43,10,2.42,IN,1675307534
580,580,fernie,49.5000,-115.0687,-8.52,97,100,0.72,CA,1675307396
581,581,san francisco,37.7749,-122.4194,11.92,64,20,1.54,US,1675307285
582,582,gornozavodsk,58.3758,58.3211,-21.04,100,82,1.24,RU,1675307535


In [196]:
humidity= city_data_df["Humidity"].astype(float)
map_plot1 = city_data_df.hvplot.points("Lng", "Lat", geo=True, legend=True, color= "r", tiles="OSM",frame_width= 700, frame_height= 500, size= humidity, xlim=(-165,165), ylim=(-60,85), alpha=0.2)
map_plot1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (_size)

In [167]:
vacation_df = city_data_df.loc[(city_data_df["Max Temp"]>22)&
                              (city_data_df["Max Temp"]<27)&
                              (city_data_df["Cloudiness"]==0)&
                              (city_data_df["Wind Speed"]<10)].reset_index(drop=True)
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,129,santiago,-33.4569,-70.6483,25.47,39,0,4.12,CL,1675307284
1,163,pisco,-13.7000,-76.2167,23.03,88,0,5.66,PE,1675307057
2,238,road town,18.4167,-64.6167,25.05,74,0,7.95,VG,1675307246
3,249,cockburn town,21.4612,-71.1419,25.33,70,0,9.14,TC,1675307432
4,308,lima,-12.0432,-77.0282,25.50,67,0,5.66,PE,1675307447
5,317,pointe michel,15.2500,-61.3833,24.09,74,0,4.33,DM,1675307133
6,351,sarh,9.1429,18.3923,24.59,19,0,2.63,TD,1675307460
7,469,san ignacio,17.1561,-89.0714,23.08,83,0,4.12,BZ,1675307500
8,539,zaraza,9.3503,-65.3245,23.03,78,0,4.53,VE,1675307523
9,566,el tigre,8.8875,-64.2454,23.43,51,0,4.93,VE,1675307530


In [168]:
hotel_df = vacation_df.loc[:,["City","Country","Lat","Lng","Humidity"]]
hotel_df.insert(5,"Hotel Name", "")
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,santiago,CL,-33.4569,-70.6483,39,
1,pisco,PE,-13.7000,-76.2167,88,
2,road town,VG,18.4167,-64.6167,74,
3,cockburn town,TC,21.4612,-71.1419,70,
4,lima,PE,-12.0432,-77.0282,67,
5,pointe michel,DM,15.2500,-61.3833,74,
6,sarh,TD,9.1429,18.3923,19,
7,san ignacio,BZ,17.1561,-89.0714,83,
8,zaraza,VE,9.3503,-65.3245,78,
9,el tigre,VE,8.8875,-64.2454,51,


In [195]:
radius= 10000
city = row["City"] + "," + row["Country"]
params= {
        "categories": "accommodation.hotel",
        "apiKey": geoapify_key}
print("Starting hotel search")
for index, row in hotel_df.iterrows():  
    lat = row["Lat"]
    lng = row["Lng"]
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    base_url = "https://api.geoapify.com/v2/places"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
hotel_df
    

Starting hotel search
santiago - nearest hotel: apart arturo prat
pisco - nearest hotel: La Portada
road town - nearest hotel: No hotel found
cockburn town - nearest hotel: The Salt Raker Inn
lima - nearest hotel: Hotel Peach Garden
pointe michel - nearest hotel: Sibouli Valley
sarh - nearest hotel: فندق سفاري
san ignacio - nearest hotel: Rainforest Haven Inn
zaraza - nearest hotel: No hotel found
el tigre - nearest hotel: Hotel La Llovizna


,City,Country,Lat,Lng,Humidity,Hotel Name
0,santiago,CL,-33.4569,-70.6483,39,apart arturo prat
1,pisco,PE,-13.7000,-76.2167,88,La Portada
2,road town,VG,18.4167,-64.6167,74,No hotel found
3,cockburn town,TC,21.4612,-71.1419,70,The Salt Raker Inn
4,lima,PE,-12.0432,-77.0282,67,Hotel Peach Garden
5,pointe michel,DM,15.2500,-61.3833,74,Sibouli Valley
6,sarh,TD,9.1429,18.3923,19,فندق سفاري
7,san ignacio,BZ,17.1561,-89.0714,83,Rainforest Haven Inn
8,zaraza,VE,9.3503,-65.3245,78,No hotel found
9,el tigre,VE,8.8875,-64.2454,51,Hotel La Llovizna


In [204]:
hotel_plot = hotel_df.hvplot.points("Lng", "Lat", geo=True, legend=True, color= "r", tiles="OSM",frame_width= 700, frame_height= 500, hover_cols=["Hotel Name"], xlim=(-100,50), ylim=(-60,50), size="Humidity", alpha=0.2)
hotel_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name)